# 8 Declaring types and classes

This IHaskell notebook outlines a chapter from the 2nd edition of *Programming in Haskell* by Prof. Graham Hutton,
demonstrating examples and summarising some of the sections.

## 8.1 Type declarations

introduces a new name (or synonym) for an existing type.
```haskell
type String = [Char]
```

In [1]:
type Pos = (Int, Int)

type Trans = Pos -> Pos

Type declarations cannot be recursive.

In [2]:
type Tree = (Int, [Tree])

<interactive>:1:1: error: Cycle in type synonym declarations: <interactive>:1:1-25: type Tree = (Int, [Tree])

can be parameterised by other types.

In [3]:
type Pair a = (a,a)

can be parameterised by more than one parameter

In [4]:
type Assoc k v = [(k,v)]

In [5]:
find :: Eq k => k -> Assoc k v -> [v]
find k table = [v | (k',v) <- table, k == k']

In [6]:
find 'b' [('a',1),('b',2),('c',3),('b',4)]
find 'c' [('a',1),('b',2),('c',3),('b',4)]
find 'x' [('a',1),('b',2),('c',3),('b',4)]

[2,4]

[3]

[]

## 8.2 Data declarations

declares a completely new type, not synnonyms of existing types.
```haskell
data Bool = False | True
```
The type `Bool` comprises two new values, named `False` and `True`.
Such names assigned by the programmer to create new values of the type are called *data constructors*, or simply *constructors*.

In [7]:
data Move = North | South | East | West

In [8]:
North

<interactive>:1:1: error:
    • No instance for (Show Move) arising from a use of ‘print’
    • In a stmt of an interactive GHCi command: print it

In [9]:
data Move = North | South | East | West  deriving Show

In [10]:
North
South
East
West

North

South

East

West

In [11]:
move :: Move -> Pos -> Pos
move North (x,y) = (x,y+1)
move South (x,y) = (x,y-1)
move East  (x,y) = (x+1,y)
move West  (x,y) = (x-1,y)

In [12]:
move :: Move -> Trans
move North = \(x,y) -> (x,y+1)
move South = \(x,y) -> (x,y-1)
move East  = \(x,y) -> (x+1,y)
move West  = \(x,y) -> (x-1,y)

In [13]:
moves :: [Move] -> Pos -> Pos
moves []     p = p
moves (m:ms) p = moves ms (move m p)

In [14]:
moves :: [Move] -> Trans
moves []     = id
moves (m:ms) = moves ms . move m

In [15]:
moves :: [Move] -> Trans
moves = foldr (.) id . map move . reverse

In [16]:
moves [North,West] (3,3)
moves [South,East] (3,3)
moves [North,West,South,East] (3,3)

(2,4)

(4,2)

(3,3)

Data constructors can have some arguments.

In [17]:
data Shape = Circle Float | Rect Float Float  deriving Show

In [18]:
:type Circle
:type Rect

Circle :: Float -> Shape

Rect :: Float -> Float -> Shape

In [19]:
{-# LANGUAGE GADTs #-}
data Shape where
  Circle :: Float -> Shape
  Rect :: Float -> Float -> Shape
  deriving Show

In [20]:
area :: Shape -> Float
area (Circle r) = pi * r^2
area (Rect x y) = x * y

Data declarations can also be parameterised.
```haskell
data Maybe a = Nothing | Just a
```

In [21]:
lookup :: Eq k => k -> Assoc k v -> Maybe v
lookup k []                        = Nothing
lookup k ((k',v) : ps) | k == k'   = Just v
                       | otherwise = lookup k ps

In [22]:
lookup 'b' [('a',1),('b',2),('c',3),('b',4)]
lookup 'c' [('a',1),('b',2),('c',3),('b',4)]
lookup 'x' [('a',1),('b',2),('c',3),('b',4)]

Just 2

Just 3

Nothing

## 8.3 Newtype declarations

## 8.4 Recursive types

In [23]:
data Nat = Z | S Nat  deriving Show

In [24]:
nat2int :: Nat -> Int
nat2int Z     = 0
nat2int (S n) = 1 + nat2int n

int2nat :: Int -> Nat
int2nat 0 = Z
int2nat k = S (int2nat (k-1))

In [25]:
add :: Nat -> Nat -> Nat
add Z     n = n
add (S m) n = S (add m n)

In [26]:
data List a = Nil | Cons a (List a)  deriving Show

In [27]:
len :: List a -> Int
len Nil         = 0
len (Cons _ xs) = 1 + len xs

and there are trees ...

In [28]:
data Tree a = Null | Node a (Tree a) (Tree a)  deriving Show

In [29]:
t :: Tree Int
t = Node 5 (Node 3 (Node 1 Null Null) (Node 4 Null Null))
           (Node 7 (Node 6 Null Null) (Node 8 Null Null))

In [30]:
occurs :: Eq a => a -> Tree a -> Bool
occurs x Null           = False
occurs x (Node y t1 t2) = x == y || occurs x t1 || occurs x t2

In [31]:
inorder :: Tree a -> [a]
inorder Null           = []
inorder (Node x t1 t2) = inorder t1 ++ [x] ++ inorder t2

In [32]:
inorder t

[1,3,4,5,6,7,8]

## 8.5 Class and instance declarations

In [30]:
-- type Queue a = ([a], [a])

-- 새로운 타입으로
data Queue a = Q [a] [a]
-- deriving Eq
-- deriving Show

-- 직접 타입 클래스 인스턴스 정의
instance Show a => Show (Queue a) where
  -- show :: Queue a -> String
  show (Q xs ys) = "list2Queue " ++ show(xs ++ reverse ys)

list2Queue xs = Q xs []

instance Eq a => Eq (Queue a) where
  Q xs1 ys1 == Q xs2 ys2
    = (xs1 ++ reverse ys1) == (xs2 ++ reverse ys2) 

-- (/=) 는 정의하지 않아도 디폴트로 정의됨
-- 찾아보기: Java interface에서 default 메소드

-- 큐에 원소 넣기
enque :: a -> Queue a -> Queue a
enque x (Q xs ys) = Q (x:xs) ys

-- 큐에서 원소 빼기
deque :: Queue a -> (a, Queue a)
deque (Q [] [])     = error "empty Queue"
deque (Q xs (y:ys)) = (y, Q xs ys)
deque (Q xs [])     = deque (Q [] (reverse xs))

emptyQ = Q [] []

In [18]:
:type show

show :: forall a. Show a => a -> String

In [21]:
enque 1 emptyQ
enque 2 it
enque 3 it
deque it
enque 4 (snd it)

list2Queue [1]

list2Queue [2,1]

list2Queue [3,2,1]

(1,list2Queue [3,2])

list2Queue [4,3,2]

In [31]:
Q [1,2] [4,3]
Q [1,2,3] [4]
Q [1] [4,3,2]

Q [1] [4,3,2] == Q [1] [4,3,2]
Q [1,2] [4,3] == Q [1] [4,3,2]

list2Queue [1,2,3,4]

list2Queue [1,2,3,4]

list2Queue [1,2,3,4]

True

True

## 8.6 Tautalogy checker

In [33]:
type Assoc k v = [(k,v)]

In [34]:
find :: Eq k => k -> Assoc k v -> [v]
find k table = [v | (k',v) <- table, k == k']

In [35]:
find 'b' [('a',1),('b',2),('c',3),('b',4)]
find 'c' [('a',1),('b',2),('c',3),('b',4)]
find 'x' [('a',1),('b',2),('c',3),('b',4)]

[2,4]

[3]

[]

Data declarations can also be parameterised.
```haskell
data Maybe a = Nothing | Just a
```

In [36]:
lookup :: Eq k => k -> Assoc k v -> Maybe v
lookup k []                        = Nothing
lookup k ((k',v) : ps) | k == k'   = Just v
                       | otherwise = lookup k ps

In [37]:
lookup 'b' [('a',1),('b',2),('c',3),('b',4)]
lookup 'c' [('a',1),('b',2),('c',3),('b',4)]
lookup 'x' [('a',1),('b',2),('c',3),('b',4)]

Just 2

Just 3

Nothing

In [41]:
-- 무조건 한개 매칭되는 k가 있다고 가정하는 함수 (아닌 경우 에러)
-- lookup' k = head . find k

lookup' k table = v  where Just v = lookup k table 

:type lookup'

lookup' :: forall k a. Eq k => k -> Assoc k a -> a

In [42]:
lookup' 'b' [('a',1),('b',2),('c',3),('b',4)]
lookup' 'c' [('a',1),('b',2),('c',3),('b',4)]
lookup' 'x' [('a',1),('b',2),('c',3),('b',4)]

2

3

<interactive>:1:28-50: Irrefutable pattern failed for pattern Just v

In [46]:
type Nm = String

data Prop = Const Bool      -- True, False 둘 중 하나
          | Var Nm          -- A, B, C, D, 이런것들
          | Not Prop        -- ㄱ A
          | And Prop Prop   -- (A /\ B)
          | Imp Prop Prop   -- (A => B)
          deriving Show

-- Substitution 변수를 값으로 바꿔치기하기 위해 만든 테이블이라는 뜻에서
-- Valuation 변수를 값에 대응시켜 준다는 의미로
-- Environment 실행환경 (변수에 해당하는 값이 포함)
type Subst = Assoc Nm Bool  -- [(Nm,Bool)]

In [47]:
p1 = And (Var "A") (Not (Var "A"))
p2 = Imp (And (Var "A") (Var "B")) (Var "A")
p3 = Imp (Var "A") (And (Var "A") (Var "B"))
p4 = Imp (And (Var "A") (Imp (Var "A") (Var "B"))) (Var "B")

In [48]:
eval :: Prop -> Subst -> Bool 
eval (Const b) _ = b
eval (Var x)   s = lookup' x s
eval (Not p)   s = not (eval p s)
eval (And p q) s = eval p s && eval q s
eval (Imp p q) s = eval p s <= eval q s

In [49]:
False <= False
False <= True
True  <= False -- 거짓
True  <= True

True

True

False

True

In [50]:
import Data.List (union)

vars :: Prop -> [Nm]
vars (Const _) = []
vars (Var x)   = [x]
vars (Not p)   = vars p
vars (And p q) = vars p `union` vars q
vars (Imp p q) = vars p `union` vars q

In [52]:
vars p1
vars p2
vars p3
vars p4

["A"]

["A","B"]

["A","B"]

["A","B"]

In [53]:
eval p1 [("A",True)]
eval p2 [("A",True),("B",False)]
eval p3 [("A",True),("B",False)]
eval p3 [("A",True),("B",True)]

False

True

False

True

In [54]:
bools :: Int -> [[Bool]]
bools 0 = [[]]
bools n = [b:bs | b <- [False,True], bs <- bools (n-1)]

In [56]:
bools 1
bools 2
bools 3

[[False],[True]]

[[False,False],[False,True],[True,False],[True,True]]

[[False,False,False],[False,False,True],[False,True,False],[False,True,True],[True,False,False],[True,False,True],[True,True,False],[True,True,True]]

In [57]:
mapM_ print $ bools 3

[False,False,False]
[False,False,True]
[False,True,False]
[False,True,True]
[True,False,False]
[True,False,True]
[True,True,False]
[True,True,True]

In [58]:
-- 명제 p에 들어있는 모든 변수를 값에 대응시키는
-- 가능한 모든 방법을 다 만들어주는 함수
substs :: Prop -> [Subst]
substs p = [zip vs s | s <- bools (length vs)] 
         where vs = vars p

In [59]:
substs p1
substs p2
substs p3
substs p4

[[("A",False)],[("A",True)]]

[[("A",False),("B",False)],[("A",False),("B",True)],[("A",True),("B",False)],[("A",True),("B",True)]]

[[("A",False),("B",False)],[("A",False),("B",True)],[("A",True),("B",False)],[("A",True),("B",True)]]

[[("A",False),("B",False)],[("A",False),("B",True)],[("A",True),("B",False)],[("A",True),("B",True)]]

In [60]:
isTaut :: Prop -> Bool
isTaut p = and [eval p s | s <- substs p]

In [61]:
isTaut :: Prop -> Bool
isTaut p = all (eval p) (substs p)

* `p1`: $~ A \land \lnot A$
* `p2`: $~ (A \land B) \Rightarrow A$
* `p3`: $~ A \Rightarrow (A \land B)$
* `p4`: $~(A \Rightarrow (A \Rightarrow B)) \Rightarrow B$

In [62]:
isTaut p1
isTaut p2
isTaut p3
isTaut p4

False

True

False

True

## 8.7 Abstract machine

----

In [49]:
:opt no-pager -- to inline :info